<a href="https://colab.research.google.com/github/K-107/exercise/blob/main/7_The_functional_API_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 함수형 API

In [1]:
from keras import Model, Input, layers
# from keras import Sequential은 필요가 없다!

# 함수형 API
input_tensor = Input(shape=(64,))
x = layers.Dense(32, activation='relu')(input_tensor)
x = layers.Dense(32, activation='relu')(x)
output_tensor = layers.Dense(10, activation='softmax')(x)

model = Model(input_tensor, output_tensor)

model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 64)]              0         
_________________________________________________________________
dense (Dense)                (None, 32)                2080      
_________________________________________________________________
dense_1 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_2 (Dense)              (None, 10)                330       
Total params: 3,466
Trainable params: 3,466
Non-trainable params: 0
_________________________________________________________________


In [2]:
model.compile(optimizer='adam', loss='categorical_crossentropy')

import numpy as np
x_train = np.random.random((1000, 64))
y_train = np.random.random((1000, 10))

model.fit(x_train, y_train, epochs=10, batch_size=128)
socre = model.evaluate(x_train, y_train)

Epoch 1/10
8/8 [==============================] - 1s 2ms/step - loss: 12.3538
Epoch 2/10
8/8 [==============================] - 0s 2ms/step - loss: 12.8679
Epoch 3/10
8/8 [==============================] - 0s 2ms/step - loss: 13.7206
Epoch 4/10
8/8 [==============================] - 0s 2ms/step - loss: 14.8097
Epoch 5/10
8/8 [==============================] - 0s 2ms/step - loss: 16.3541
Epoch 6/10
8/8 [==============================] - 0s 2ms/step - loss: 19.2004
Epoch 7/10
8/8 [==============================] - 0s 2ms/step - loss: 23.7912
Epoch 8/10
8/8 [==============================] - 0s 2ms/step - loss: 29.3482
Epoch 9/10
8/8 [==============================] - 0s 2ms/step - loss: 35.5823
Epoch 10/10
32/32 [==============================] - 0s 924us/step - loss: 48.8352


# 2개의 입력을 가진 질문-응답 모델의 함수형 API

In [3]:
from keras import Input, layers, Model
'''
import하는 순서를 보자면 먼저 Input값을 받고, layers를 만든 다음 Model로 묶어준다!
'''

text_vocabulary_size = 10000
question_vocabulary_size = 10000
answer_vocabulary_size = 500

text_input = Input(shape=(None,), dtype='int32', name='text')
embedded_text = layers.Embedding(text_vocabulary_size, 64)(text_input)
encoded_text = layers.SimpleRNN(32)(embedded_text)

question_input = Input(shape=(None,), dtype='int32', name='question')
embedded_question = layers.Embedding(text_vocabulary_size, 32)(question_input)
encoded_question = layers.SimpleRNN(16)(embedded_question)

concatenated = layers.concatenate([encoded_text, encoded_question], axis=-1)

answer = layers.Dense(answer_vocabulary_size, activation='softmax')(concatenated)

model = Model([text_input, question_input], answer)

model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text (InputLayer)               [(None, None)]       0                                            
__________________________________________________________________________________________________
question (InputLayer)           [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, None, 64)     640000      text[0][0]                       
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 32)     320000      question[0][0]                   
____________________________________________________________________________________________

In [4]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])

import numpy as np
# from keras.utils import to_categorical # 코드가 삭제된 듯!
from keras.utils.np_utils import to_categorical

num_samples = 1000
max_length = 100

text = np.random.randint(1, text_vocabulary_size, size=(num_samples, max_length))
question = np.random.randint(1, question_vocabulary_size, size=(num_samples, max_length))

answers = np.random.randint(0, answer_vocabulary_size, size=num_samples)

answers = to_categorical(answers)

model.fit([text, question], answers, epochs=10, batch_size=128)

# model.fit({'text':text, 'question':question}, answers, epochs=10, batch_size=128)

Epoch 1/10
8/8 [==============================] - 2s 65ms/step - loss: 6.2164 - acc: 0.0020
Epoch 2/10
8/8 [==============================] - 1s 65ms/step - loss: 6.0006 - acc: 0.4470
Epoch 3/10
8/8 [==============================] - 1s 73ms/step - loss: 5.7866 - acc: 0.8970
Epoch 4/10
8/8 [==============================] - 1s 63ms/step - loss: 5.5660 - acc: 0.9950
Epoch 5/10
8/8 [==============================] - 1s 63ms/step - loss: 5.3521 - acc: 1.0000
Epoch 6/10
8/8 [==============================] - 0s 62ms/step - loss: 5.1467 - acc: 1.0000
Epoch 7/10
8/8 [==============================] - 1s 64ms/step - loss: 4.9548 - acc: 1.0000
Epoch 8/10
8/8 [==============================] - 1s 63ms/step - loss: 4.7789 - acc: 1.0000
Epoch 9/10
8/8 [==============================] - 0s 62ms/step - loss: 4.6119 - acc: 1.0000
Epoch 10/10
8/8 [==============================] - 0s 61ms/step - loss: 4.4467 - acc: 1.0000


In [5]:
# 이건 된다.

import tensorflow as tf
tf.keras.utils.to_categorical

<function keras.utils.np_utils.to_categorical>

참고! SimpleRNN의 파라미터 수 계산

https://ichi.pro/ko/keras-ui-rnn-mich-lstme-daehan-sil-yongjeog-in-gaideu-167190584259004

In [6]:
from keras.models import Sequential
from keras.layers import SimpleRNN

model = Sequential()
model.add(SimpleRNN(3, input_shape=(2,10)))
# model.add(SimpleRNN(3, input_length=2, input_dim=10))와 동일함.
model.summary()

# 3*3+3*10+3 = 42

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_2 (SimpleRNN)     (None, 3)                 42        
Total params: 42
Trainable params: 42
Non-trainable params: 0
_________________________________________________________________


# 다중 출력 모델

In [11]:
from keras import Input, layers, Model

vocabulary_size = 50000
num_income_groups = 10

posts_input = Input(shape=(None,), dtype='int32', name='posts')
embedded_posts = layers.Embedding(vocabulary_size, 256)(posts_input)
x = layers.Conv1D(128, 5, activation='relu')(embedded_posts)
x = layers.MaxPooling1D(5)(x) # strides: Integer, or None. Specifies how much the pooling window moves
                           # for each pooling step. If None, it will default to pool_size.
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dense(128, activation='relu')(x)

age_prediction = layers.Dense(1, name='age')(x)
income_prediction = layers.Dense(num_income_groups, activation='softmax', name='income')(x)
gender_prediction = layers.Dense(1, activation='sigmoid', name='gender')(x)

model = Model(posts_input, [age_prediction, income_prediction, gender_prediction])

model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
posts (InputLayer)              [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding_6 (Embedding)         (None, None, 256)    12800000    posts[0][0]                      
__________________________________________________________________________________________________
conv1d_16 (Conv1D)              (None, None, 128)    163968      embedding_6[0][0]                
__________________________________________________________________________________________________
max_pooling1d_7 (MaxPooling1D)  (None, None, 128)    0           conv1d_16[0][0]                  
____________________________________________________________________________________________

In [14]:
# model.compile(optimizer='adam', loss=['mse', 'categorical_crossentropy', 'binary_crossentropy'])
# model.compile(optimizer='adam',
#               loss={'age':'mse', 'income':'categorical_crossentropy', 'gender':'binary_crossentropy'})

model.compile(optimizer='adam',
              loss=['mse', 'categorical_crossentropy', 'binary_crossentropy'],
              loss_weights=[0.25, 1., 10.,])
# model.compile(optimizer='adam',
#               loss={'age':'mse', 'income':'categorical_crossentropy', 'gender':'binary_crossentropy'},
#               loss_weights={'age': 0.25,
#                             'income': 1.,
#                             'gender': 10.})

In [ ]:
model.fit('posts':post, [age_targets, income_targets, gender_targets],
          epochs=10, batch_size=64)

# model.fit(posts, {'age': age_targets,
#                   'income':income_targets,
#                   'gender': gender_targets},
#           epochs=10, batch_size=64)